# Messenger Radio-Science collection

Select a subset from the MESSENGER Radio Science RSR Data Products Collection and gather the data in a single dataframe for easy analysis and visualization.  

In [ ]:
import pds.peppi as peppi
import pandas as pd
import pdr
import os
from urllib.request import urlretrieve
from datetime import datetime

## Get the collection's product subset

From its identifier found on https://pds.nasa.gov/

In [ ]:
client = peppi.PDSRegistryClient()
start_date = datetime.fromisoformat("2012-03-02")
end_date = datetime.fromisoformat("2012-03-05")
products = peppi.Products(client).of_collection("urn:nasa:pds:mess-rs-raw:data-rsr::1.0").after(start_date).before(end_date)

## Iterate over the data products

Iterate over the products maching the the criteria above and aggregate the data content into a single dataframe, using [PDR](https://pdr.readthedocs.io/) .

In [ ]:
df = None
for i, p in enumerate(products):

    # download label
    remote_label_path = p.properties['ops:Label_File_Info.ops:file_ref'][0]
    local_label_path = os.path.basename(remote_label_path)
    urlretrieve(remote_label_path, local_label_path)
 
    # download product
    remote_data_path = p.properties['ops:Data_File_Info.ops:file_ref'][0]
    local_data_path = os.path.basename(remote_data_path)
    urlretrieve(remote_data_path, local_data_path)
    
    # use Planetary Data Reader to read the data into a pandas DataFrame
    data = pdr.read(os.path.abspath(local_label_path))
    
    # append current data to a main dataframe
    df = pd.concat([df, data[ 'MESSENGER_Radio_Science_Receiver_(RSR)_raw_data' ]])
    
    # break because this is a lot of data
    if i>3:
        break

In [ ]:
df['I/Q Pair, I_1']

## Plot one random field

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
plt.plot(df["I/Q Pair, I_1"])
plt.show()

In [ ]:
plt.clf()